In [79]:
import locale
import sqlite3

# from visualize import viztool
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.dates as mdates
import numpy as np
from scipy import stats
import seaborn as sns

# Set locale to German
locale.setlocale(locale.LC_TIME, "de_DE.utf8")

# Apply the default theme
sns.set_theme()


# count the number of rows in the database
def read_from_db(city):
    conn = sqlite3.connect(f"bachelorarbeit-data/{city.lower()}.db")

    slots = pd.read_sql_query("SELECT * FROM Slots;", conn, parse_dates=["timeslot"])

    availabilities = pd.read_sql_query(
        "SELECT * FROM Availabilities;", conn, parse_dates=["available", "taken"]
    )

    conn.close()

    return slots, availabilities



duesseldorf_slots, duesseldorf_availabilities = read_from_db("Duesseldorf")
dresden_slots, dresden_availabilities = read_from_db("Dresden")
kiel_slots, kiel_availabilities = read_from_db("Kiel")

all_slots = pd.concat([duesseldorf_slots, dresden_slots, kiel_slots])
all_availabilities = pd.concat(
    [duesseldorf_availabilities, dresden_availabilities, kiel_availabilities]
)

def preprocess_dataframe(df):

    # add count of availabilities per s_id
    df["count_availabilities"] = df.groupby("s_id")["a_id"].transform("count")

    # lose precision to only minutes
    df["timeslot"] = df["timeslot"].dt.floor("min")
    df["available"] = df["available"].dt.floor("min")
    df["taken"] = df["taken"].dt.floor("min")

    # add weekday
    df["weekday"] = df["timeslot"].dt.day_name()

    # add hour
    df["hour"] = df["timeslot"].dt.hour

    # timedelta between available and taken
    df["delta"] = df["taken"] - df["available"]

    # time until slot
    df["time_until_slot"] = df["timeslot"] - df["taken"]

    # add total delta per s_id
    df["total_delta"] = df.groupby("s_id")["delta"].transform("sum")

    # sort by total delta
    df = df.sort_values("s_id", ascending=True)

    return df

def join_all_data():

    sql = """
        SELECT 
            Availabilities.slot_id as s_id,
            Slots.office as office,
            Slots.city as city,
            Slots.timeslot as timeslot,
            Slots.concern as concern,
            Availabilities.id as a_id,
            Availabilities.available as available,
            Availabilities.taken as taken
        FROM Slots
        JOIN Availabilities ON Slots.id = Availabilities.slot_id;
        """
    
    
    df = pd.DataFrame()
    for city in ["Duesseldorf", "Dresden", "Kiel"]:
        conn = sqlite3.connect(f"bachelorarbeit-data/{city.lower()}.db")
        df = pd.concat([df, pd.read_sql_query(sql, conn, parse_dates=["timeslot", "available", "taken"])])
        conn.close()

    df = preprocess_dataframe(df)
    return df


In [80]:
all_slots

,id,office,city,timeslot,concern
0,1,Bilk,Duesseldorf,2024-11-08 09:35:00,Anmeldung
1,2,Bilk,Duesseldorf,2024-11-22 07:30:00,Anmeldung
2,3,Bilk,Duesseldorf,2024-11-22 07:35:00,Anmeldung
3,4,Bilk,Duesseldorf,2024-11-22 07:40:00,Anmeldung
4,5,Bilk,Duesseldorf,2024-11-22 07:45:00,Anmeldung
...,...,...,...,...,...
40681,40682,Suchsdorf,Kiel,2025-01-07 16:15:00,Anmeldung
40682,40683,Suchsdorf,Kiel,2025-01-07 16:20:00,Anmeldung
40683,40684,Suchsdorf,Kiel,2025-01-07 16:25:00,Anmeldung
40684,40685,Suchsdorf,Kiel,2025-01-07 16:30:00,Anmeldung


In [81]:
all_slots.agg(
    {
        "id": ["count", "nunique", "min", "max"],
        "timeslot": ["min", "max"],
    }
)

,id,timeslot
count,214321,NaT
nunique,119215,NaT
min,1,2024-11-08 09:35:00
max,119215,2025-02-03 15:45:00


In [82]:
all_availabilities.agg(
    {
        "id": ["count", "nunique", "min", "max"],
        "slot_id": ["count", "nunique", "min", "max"],
        "available": ["min", "max"],
        "taken": ["min", "max"],
    }
)

,id,slot_id,available,taken
count,1599791,1599791,NaT,NaT
nunique,1001220,119215,NaT,NaT
min,1,1,2024-11-08 09:27:49.872613,2024-11-08 09:29:33.346158
max,1001220,119215,2024-12-10 01:06:58.446959,2024-12-10 01:10:12.528407


In [83]:
all_slots.groupby(["concern"])[["id"]].count()

,id
concern,
Abmeldung,35028
Anmeldung,44268
Personalausweis - Antrag,45042
Reisepass - Antrag,43447
Ummeldung,46536


In [84]:
all_slots.groupby(["city", "concern"])[["id"]].count()

id
city        concern                        
Dresden     Abmeldung                 10462
            Anmeldung                 10086
            Personalausweis - Antrag  11756
            Reisepass - Antrag        10753
            Ummeldung                 11363
Duesseldorf Abmeldung                 24566
            Anmeldung                 23295
            Personalausweis - Antrag  23644
            Reisepass - Antrag        23481
            Ummeldung                 24229
Kiel        Anmeldung                 10887
            Personalausweis - Antrag   9642
            Reisepass - Antrag         9213
            Ummeldung                 10944

In [85]:
all_slots.groupby(["city", "office"])[["id"]].count()

id
city        office                       
Dresden     Altstadt                 4839
            Blasewitz                4023
            Cossebaude               4890
            Cotta                    4319
            Junioramt                3497
            Klotzsche                3613
            Langenbrueck             3487
            Leuben                   4386
            Neustadt                 4044
            Pieschen                 5154
            Plauen                   3964
            Prohlis                  4236
            Weixdorf                 3968
Duesseldorf Benrath                 12215
            Bilk                    11869
            Dienstleistungszentrum  12544
            Eller                   12045
            Garath                  10653
            Gerresheim              11340
            Kaiserswerth            11099
            Oberkassel              11183
            Rath                    12044
            Unterbach                1897
            Wersten/Holthausen      12326
Kiel        Dietrichsdorf            5793
            Elmschenhagen            5928
            Hassee                   5801
            Mettenhof                6040
            Pries                    6100
            Rathaus                  9235
            Suchsdorf                1789

In [86]:
df = join_all_data()

In [88]:
df

,s_id,office,city,timeslot,concern,a_id,available,taken,count_availabilities,weekday,hour,delta,time_until_slot,total_delta
16612,1,Dietrichsdorf,Kiel,2024-11-11 08:25:00,Personalausweis - Antrag,16613,2024-11-09 07:17:00,2024-11-11 08:23:00,4,Monday,8,2 days 01:06:00,0 days 00:02:00,2 days 23:01:00
0,1,Dietrichsdorf,Kiel,2024-11-11 08:25:00,Personalausweis - Antrag,1,2024-11-08 09:29:00,2024-11-09 07:15:00,4,Monday,8,0 days 21:46:00,2 days 01:10:00,2 days 23:01:00
0,1,Cotta,Dresden,2024-11-08 10:30:00,Anmeldung,1,2024-11-08 09:28:00,2024-11-08 09:32:00,4,Friday,10,0 days 00:04:00,0 days 00:58:00,2 days 23:01:00
0,1,Bilk,Duesseldorf,2024-11-08 09:35:00,Anmeldung,1,2024-11-08 09:27:00,2024-11-08 09:32:00,4,Friday,9,0 days 00:05:00,0 days 00:03:00,2 days 23:01:00
1,2,Dietrichsdorf,Kiel,2024-11-11 08:30:00,Personalausweis - Antrag,2,2024-11-08 09:29:00,2024-11-09 07:15:00,7,Monday,8,0 days 21:46:00,2 days 01:15:00,3 days 00:45:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001198,119211,Benrath,Duesseldorf,2024-12-19 16:45:00,Anmeldung,1001199,2024-12-10 01:05:00,NaT,1,Thursday,16,NaT,NaT,0 days 00:00:00
1001205,119212,Benrath,Duesseldorf,2024-12-19 16:30:00,Reisepass - Antrag,1001206,2024-12-10 01:06:00,NaT,1,Thursday,16,NaT,NaT,0 days 00:00:00
1001206,119213,Benrath,Duesseldorf,2024-12-19 16:35:00,Reisepass - Antrag,1001207,2024-12-10 01:06:00,NaT,1,Thursday,16,NaT,NaT,0 days 00:00:00
1001207,119214,Benrath,Duesseldorf,2024-12-19 16:40:00,Reisepass - Antrag,1001208,2024-12-10 01:06:00,NaT,1,Thursday,16,NaT,NaT,0 days 00:00:00


In [115]:
df.groupby(["city", "office"])[["s_id", "a_id"]].nunique()
df_grouped = df.groupby(["city", "office"])[["s_id", "a_id"]].nunique()
df_grouped["ratio"] = (df_grouped["a_id"] / df_grouped["s_id"]).round(2)
df_grouped = df_grouped.sort_values(by=["city", "office", "ratio"], ascending=False)
df_grouped

s_id    a_id  ratio
city        office                                      
Kiel        Suchsdorf                1789    2899   1.62
            Rathaus                  9235   32259   3.49
            Pries                    6100   21261   3.49
            Mettenhof                6040   22731   3.76
            Hassee                   5801   22254   3.84
            Elmschenhagen            5928   17775   3.00
            Dietrichsdorf            5793   20013   3.45
Duesseldorf Wersten/Holthausen      12326   55709   4.52
            Unterbach                1897    3432   1.81
            Rath                    12044   40666   3.38
            Oberkassel              11183   25855   2.31
            Kaiserswerth            11099   30807   2.78
            Gerresheim              11340   30937   2.73
            Garath                  10653   36410   3.42
            Eller                   12045   39091   3.25
            Dienstleistungszentrum  12544  666957  53.17
            Bilk                    11869   30322   2.55
            Benrath                 12215   41034   3.36
Dresden     Weixdorf                 3968   14161   3.57
            Prohlis                  4236   35486   8.38
            Plauen                   3964   35264   8.90
            Pieschen                 5154   50839   9.86
            Neustadt                 4044   38319   9.48
            Leuben                   4386   25229   5.75
            Langenbrueck             3487   18698   5.36
            Klotzsche                3613   26871   7.44
            Junioramt                3497   26066   7.45
            Cotta                    4319   38771   8.98
            Cossebaude               4890   57909  11.84
            Blasewitz                4023   34466   8.57
            Altstadt                 4839   57300  11.84